In [1]:
from itertools import cycle, count, islice, chain, repeat, takewhile, groupby

In [2]:
with open('../data/2024/day9.txt') as f:
    data = f.read()

In [3]:
blocks = []
modes = cycle(('file','free'))
file_ids = chain.from_iterable((i, i) for i in count())

# Decompress (2333133121414131402 -> 00...111...2...333.44.5555.6666.777.888899)
for digit, mode, file_id in zip(data, islice(modes, len(data)), islice(file_ids, len(data))):
    blocks.append(list(repeat(str(file_id), int(digit)) if mode == 'file' else repeat('.', int(digit))))

In [4]:
# Part 1
part1_blocks = list(chain(*blocks))
left, right = 0, len(part1_blocks)-1

# Iterate from both ends simultaneously until the pointers cross
while True:
    while left < len(part1_blocks) and part1_blocks[left] != '.': left += 1
    while right >= 0 and part1_blocks[right] == '.': right -= 1
    if left > right: break
    # Swap our leftmost free space with rightmost file
    part1_blocks[right], part1_blocks[left] = part1_blocks[left], part1_blocks[right]

print('Part 1:',
      sum([i * int(file_id) for i, file_id in enumerate(takewhile(lambda c: c != '.', part1_blocks))]))

Part 1: 6463499258318


In [5]:
# Part 2
part2_blocks = list(chain(*blocks))

# Tuples of (file_id or '.', index start, index end, length)
chunks = [
    (key, indices[0], indices[-1], indices[-1] - indices[0] + 1)
    for key, index_groups in groupby(enumerate(part2_blocks), key=lambda x: x[1])
    for indices, group in [([idx for idx,_ in index_groups], [v for _,v in index_groups])]
]

# Split chunks into (reversed) files and free space
files = list(filter(lambda chunk: chunk[0] != '.', reversed(chunks)))
frees = list(filter(lambda chunk: chunk[0] == '.', chunks))

# Move rightmost files into leftmost free space
for file in files:
    for free_n, free in enumerate(frees):
        # Only move leftward
        if free[1] > file[1]:
            break
        # Is this free space block big enough?
        elif free[3] >= file[3]:
            part2_blocks[free[1]:free[1]+file[3]], part2_blocks[file[1]:file[2]+1] = \
                part2_blocks[file[1]:file[2]+1], part2_blocks[free[1]:free[1]+file[3]]
            # If the exact size, use it all
            if free[3] == file[3]:
                del frees[free_n]
            # If we only need part of this free space block, consume from left and leave a remainder
            else:
                used_len = file[3]
                frees[free_n] = (free[0], free[1]+used_len, free[2], free[3]-used_len)
            break

# Sum using full ids (e.g. 100) not characters (1,0,0)
print('Part 2:',
      sum([(i * (0 if '.' == file_id else int(file_id))) for i, file_id in enumerate(part2_blocks)]))

Part 2: 6493634986625
